In [1]:
import torch
print(torch.__version__)

2.2.1


In [2]:
# 数据集有10类，32x32，文件名例如0_12345.jpg ,一共6w张
# 数据下载：https://www.cs.toronto.edu/~kriz/cifar.html
# 官网贴心的提供的python格式的读取函数，尽管类型是字典类
# 温馨提示只能读取data_batch_1这样的，下载下来是tar.gz,需要全部解压
# 这样的读取数据有一点点麻烦，原作者github上提供了更简单的方式，不过下载起来太麻烦了
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
cifar10 = unpickle('D:\\datasets\\cifar-10-batches-py\\data_batch_1')
#data是矩阵，labels是列表  32 x 32 x 3 = 3072 
cifar10[b'data'].shape,len(cifar10[b'labels'])

In [ ]:
def load_data(path):
    data = unpickle(path)
    x = torch.Tensor(data[b'data']).reshape(10000,3,32,32) / 255
    y = torch.Tensor(data[b'labels'])
    return x,y

#其实都已经是10000x3x32x32 ，本来就分批次好了，结果又要重新打回去，又要重新loader
class DataSet(torch.utils.data.Dataset):
    def __init__(self):
        x1,y1 = load_data('D:\datasets\cifar-10-batches-py\data_batch1')
        x2,y2 = load_data('D:\datasets\cifar-10-batches-py\data_batch2')
        x3,y3 = load_data('D:\datasets\cifar-10-batches-py\data_batch3')
        x4,y4 = load_data('D:\datasets\cifar-10-batches-py\data_batch4')
        x5,y5 = load_data('D:\datasets\cifar-10-batches-py\data_batch5')
        x = torch.cat([x1,x2,x3,x4,x5],dim=0)
        y = torch.cat([y1,y2,y3,y4,y5],dim=0)
    def __len__(self):
        return x.shape[0]
    def __getitem__(self,index):
        width,height,fat = get_rectangle()
        # x 是一维的   有两个数据
        X = torch.FloatTensor([width,height])
        # X是tensor ， 但是y不是
        Y = fat
        return X,Y
dataset = DataSet()
len(dataset),dataset[1000]

![alt text](image.png)

In [58]:
#我直接写train，直接跳过dataset和dataloader
class Model(torch.nn.Module):
    def __init__(self):
        #一定要写继承
        super().__init__()

        self.cnn1 = torch.nn.Conv2d(3,16,5,stride=2,padding=0)
        self.cnn2 = torch.nn.Conv2d(16,32,3,stride=1,padding=1)
        self.cnn3 = torch.nn.Conv2d(32,128,7,stride=1,padding=0)

        self.pool = torch.nn.MaxPool2d(2,2)
        self.relu = torch.nn.ReLU()
        self.fc = torch.nn.Linear(128,10)
    def forward(self,x):
        #8,3,32,32 -> 8,16,14,14
        x = self.cnn1(x)
        x = self.relu(x)

        #8,16,14,14 -> 8,32,14,14     
        x = self.cnn2(x)
        x = self.relu(x)

        #8,32,14,14 -> 8,32,7,7
        x = self.pool(x)  

        #8,32,7,7 -> 8,128,1,1    
        x = self.cnn3(x)
        x = self.relu(x)
        
        #flatten
        x = x.flatten(start_dim=1)

        #不需要softmax？
        return self.fc(x)


In [27]:
model = Model()
def get_shape(module, input, output):
    print(f"Input shape: {input[0].shape}")
    print(f"Output shape: {output.shape}")
# 注册forward hook来获取每一层的输入和输出形状变化
for name, layer in model.named_children():
    #print(name,layer)
    #sequential 默认是一层
    layer.register_forward_hook(get_shape)

# 开始打印每一层的变化
model(torch.randn(1,3,32,32))

Input shape: torch.Size([1, 3, 32, 32])
Output shape: torch.Size([1, 16, 14, 14])
Input shape: torch.Size([1, 16, 14, 14])
Output shape: torch.Size([1, 16, 14, 14])
torch.Size([1, 16, 14, 14]) 1
Input shape: torch.Size([1, 16, 14, 14])
Output shape: torch.Size([1, 32, 14, 14])
Input shape: torch.Size([1, 32, 14, 14])
Output shape: torch.Size([1, 32, 14, 14])
torch.Size([1, 32, 14, 14]) 2
Input shape: torch.Size([1, 32, 14, 14])
Output shape: torch.Size([1, 32, 7, 7])
torch.Size([1, 32, 7, 7]) 3
Input shape: torch.Size([1, 32, 7, 7])
Output shape: torch.Size([1, 128, 1, 1])
Input shape: torch.Size([1, 128, 1, 1])
Output shape: torch.Size([1, 128, 1, 1])
torch.Size([1, 128, 1, 1]) 4
Input shape: torch.Size([1, 128])
Output shape: torch.Size([1, 10])


tensor([[ 0.0845, -0.0472, -0.1127, -0.0803,  0.0303,  0.0019,  0.0353, -0.0810,
         -0.0297,  0.0820]], grad_fn=<AddmmBackward0>)

In [60]:
# train
def load_data(path):
    data = unpickle(path)
    x = torch.Tensor(data[b'data']).reshape(10000,3,32,32) / 255
    y = torch.Tensor(data[b'labels'])
    return x,y
def one_hot_encoding(num, num_classes):
    # 创建一个全零的向量
    one_hot = torch.zeros(num_classes)
    # 将对应位置置为1
    one_hot[num] = 1
    return one_hot

print('load data')
batch_x,batch_y = load_data('D:\\datasets\\cifar-10-batches-py\\data_batch_1')
model = Model()
def train():

    optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
    loss_fun = torch.nn.CrossEntropyLoss()
    model.train()
    for epoch in range(10):
        print(epoch)
        for x,y in zip(batch_x,batch_y):
            #torch.Size([3, 32, 32]) torch.Size([])  需要插入一个batch维度
            x = x.unsqueeze(0)
            #y需要变成【1，10】 one-hot
            y = one_hot_encoding(int(y.item()), 10)
            y = y.unsqueeze(0)
            # 计算
            out = model(x)
            loss = loss_fun(out,y)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
train()


load data
0
1
2
3
4
5
6
7
8
9


TypeError: 'function' object is not iterable

In [69]:
batch_x,batch_y = load_data('D:\\datasets\\cifar-10-batches-py\\test_batch')
@torch.no_grad()
def test():
    acc_all = 0
    for x,y in zip(batch_x,batch_y):
        x = x.unsqueeze(0)
        #y需要变成【1，10】 one-hot
        y = one_hot_encoding(int(y.item()), 10)
        y = y.unsqueeze(0)
        
        out = model(x)
        #print(out.shape,y.shape,out,y,out.argmax(dim=1),y.argmax(dim=1))
        acc = (out.argmax(dim=1) == y.argmax(dim=1)).item() / len(y)
        print(acc)
        acc_all += acc
    acc_all = acc_all / 10000
    print('final - >',acc_all)
test()

1.0
0.0
1.0
0.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
0.0
0.0
1.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
1.0
0.0
1.0
1.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
1.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
0.0
1.0
0.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
0.0
1.0
0.0
0.0
1.0
0.0
1.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
1.0
0.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
1.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0


# 总结
- 原始网站提供的cifar10已经分好了5个训练batch，1个测试batch
- 本文使用了其中一个batch作为训练集，1个测试batch作为测试集，最终效果为0.5429，同时训练过程中batchsize设为1（主要是loader难写）
- 如果是另一种数据格式：一个文件夹下全部jpg文件的话，这样写dataset，dataloader会比较好写。本文采用的方法可能是作者存储文件比较方便，索性就这样了
- 第二种方法：pytorch其实官方提供了cifar10的api，可以依靠网络下载，直接loader很方便
- 第三种方法：原作者github上有很多压缩包，全部下载后解压，又有数据集，又有模型，只是太大了 https://github.com/lansinuote/Simple_PyTorch2

In [70]:
import torch
import torchvision
import torchvision.transforms as transforms

# 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 下载训练集和测试集
trainset = torchvision.datasets.CIFAR10(root='./cifar10_dataset', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./cifar10_data', train=False, download=True, transform=transform)

# 创建数据加载器
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

# 类别标签
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# 输出训练集和测试集的大小
print('训练集大小:', len(trainset))
print('测试集大小:', len(testset))

  1%|          | 2031616/170498071 [01:59<2:45:39, 16948.74it/s]


KeyboardInterrupt: 